In [ ]:
import pandas as pd
import sqlite3
from IPython.display import display, HTML

In [ ]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

In [ ]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [ ]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




### Preprocessing data:

In [ ]:
year_preprocessing ='''
update movie
set year = cast( substr(year,-4) as integer)
'''
conn.execute(year_preprocessing)

for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    column_name = schema.name.values[1:]
    for i in column_name:
      trim_preprocessing = 'update {} set {}= TRIM({})'.format(table,i,i)
      conn.execute(trim_preprocessing)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. Your query should return director name, the movie name, and the year.

In [ ]:
query1 ='''

with leap_year_movies as
(
select m.mid mid, m.year year, m.title title
from
movie m
where ( (m.year % 4 = 0 and m.year % 100 = 0 and m.year % 400 = 0) or (m.year % 4 = 0 and m.year % 100 != 0 ) )
),

comedy_leap_year_movies as
(
select mid, year,title
from
(select gid,mid,year,title
from
leap_year_movies lym
left join 
m_genre m_g
using (mid) )
left join genre g
using (gid)
where g.name like '%Comedy%'
)

select name,title,year
from
(select pid,year,title
from
comedy_leap_year_movies
left join
m_director m_d
using (mid))
left join
person p
using (pid)
'''
 
pd.read_sql_query(query1,conn)

In [ ]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = query1
grader_1(query1)

               Name                              title  year
0      Milap Zaveri                         Mastizaade  2016
1      Danny Leiner  Harold & Kumar Go to White Castle  2004
2    Anurag Kashyap                 Gangs of Wasseypur  2012
3      Frank Coraci        Around the World in 80 Days  2004
4     Griffin Dunne             The Accidental Husband  2008
5       Anurag Basu                             Barfi!  2012
6   Gurinder Chadha                  Bride & Prejudice  2004
7        Mike Judge    Beavis and Butt-Head Do America  1996
8  Tarun Mansukhani                            Dostana  2008
9      Shakun Batra                      Kapoor & Sons  2016
CPU times: user 52.9 ms, sys: 0 ns, total: 52.9 ms
Wall time: 53.3 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [ ]:
query2 = '''

with movie_id as
(
select m.mid id
from movie m
where m.title = 'Anand'
),

cast_id as
(
select m_c.pid id
from m_cast m_c, movie_id m_id
where m_c.mid = m_id.id
)

select p.name
from person p, cast_id c
where p.pid = c.id
'''

pd.read_sql_query(query2,conn)

In [ ]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = query2
grader_2(query2)

               Name
0  Amitabh Bachchan
1     Rajesh Khanna
2    Brahm Bhardwaj
3        Ramesh Deo
4         Seema Deo
5        Dev Kishan
6       Durga Khote
7     Lalita Kumari
8      Lalita Pawar
9      Atam Prakash
CPU times: user 124 ms, sys: 2.3 ms, total: 126 ms
Wall time: 127 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [ ]:
query_less_1970 =""" 
select p.name from person p 
inner join 
(
    select mc.pid pd, mc.mid from m_cast mc 
where mc.mid
in 
(
    select mv.mid from movie mv where mv.year<1970
)
) r1 
on r1.pd=p.pid
"""
query_more_1990 =""" 
select p.name from person p 
inner join 
(
    select mc.pid pd, mc.mid from m_cast mc 
where mc.mid 
in 
(
    select mv.mid from movie mv where mv.year>1990
)
) r1 
on r1.pd=p.pid """

In [ ]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 = query_less_1970
query_more_1990 = query_more_1990 
print(grader_3a(query_less_1970, query_more_1990))

(4942, 1)
(62570, 1)
True
CPU times: user 224 ms, sys: 4.18 ms, total: 229 ms
Wall time: 231 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [ ]:
query4a = '''
select m_d.pid, count(m_d.mid)
from
m_director m_d
group by m_d.pid
'''

pd.read_sql_query(query4a,conn)

In [ ]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a =query4a
print(grader_4a(query_4a))

         PID  count(m_d.mid)
0  nm0000180               1
1  nm0000187               1
2  nm0000229               1
3  nm0000269               1
4  nm0000386               1
5  nm0000487               2
6  nm0000965               1
7  nm0001060               1
8  nm0001162               1
9  nm0001241               1
True
CPU times: user 9.53 ms, sys: 0 ns, total: 9.53 ms
Wall time: 9.37 ms


In [ ]:
query4 = '''
select name, movie_count
from person
inner join
(select m_d.pid pid, count(m_d.mid) movie_count
from
m_director m_d
group by m_d.pid
having count(m_d.mid)>9) 
using (pid)
order by movie_count desc
'''

pd.read_sql_query(query4,conn)

In [ ]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = query4
grader_4(query4)

                   Name  movie_count
0          David Dhawan           39
1          Mahesh Bhatt           35
2       Ram Gopal Varma           30
3          Priyadarshan           30
4          Vikram Bhatt           29
5  Hrishikesh Mukherjee           27
6           Yash Chopra           21
7        Shakti Samanta           19
8       Basu Chatterjee           19
9          Subhash Ghai           18
CPU times: user 16.3 ms, sys: 893 µs, total: 17.2 ms
Wall time: 17.2 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
query5aa = '''
select m.mid,p.gender,count(*) Count
from
movie m
inner join m_cast m_c
using (mid)
inner join person p
using (pid)
group by m.mid, p.gender
'''

pd.read_sql_query(query5aa,conn)

In [ ]:
query5ab = '''
select m.mid,p.gender,count(*) Count
from
movie m
inner join m_cast m_c
using (mid)
inner join person p
using (pid)
group by m.mid, p.gender
having (p.gender = 'Male') and count(p.gender = 'Male' > 0)
'''

pd.read_sql_query(query5ab,conn)

In [ ]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3))

query_5aa = query5aa

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

query_5ab = query5ab

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         MID  Gender  Count
0  tt0021594    None      1
1  tt0021594  Female      3
2  tt0021594    Male      5
3  tt0026274    None      2
4  tt0026274  Female     11
5  tt0026274    Male      9
6  tt0027256    None      2
7  tt0027256  Female      5
8  tt0027256    Male      8
9  tt0028217  Female      3
True
         MID Gender  Count
0  tt0021594   Male      5
1  tt0026274   Male      9
2  tt0027256   Male      8
3  tt0028217   Male      7
4  tt0031580   Male     27
5  tt0033616   Male     46
6  tt0036077   Male     11
7  tt0038491   Male      7
8  tt0039654   Male      6
9  tt0040067   Male     10
True
CPU times: user 394 ms, sys: 16.1 ms, total: 410 ms
Wall time: 413 ms


In [ ]:
query5a = '''
with gender_count as
(
select m.mid mid,p.gender gender,count(*) Count, m.year
from
movie m
inner join m_cast m_c
using (mid)
inner join person p
using (pid)
group by m.mid, p.gender
)

select year, count(mid) Female_Cast_Only_Movies
from
(select mid,Count,year
from gender_count
where gender='Female') g1
left join
(select mid
from gender_count
where gender = 'Male' or gender = 'None') g2
using (mid)
where g2.mid is null
group by year
'''

pd.read_sql_query(query5a,conn)

In [ ]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = query5a
grader_5a(query5a)

   year  Female_Cast_Only_Movies
0  1939                        1
1  1999                        1
2  2000                        1
3  2018                        1
CPU times: user 298 ms, sys: 20.5 ms, total: 318 ms
Wall time: 320 ms


## Q5.b --- Report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year.

In [ ]:
query5b = '''
with gender_count as
(
select m.mid mid,p.gender gender,count(*) Count, m.year year
from
movie m
inner join m_cast m_c
using (mid)
inner join person p
using (pid)
group by m.mid, p.gender
),

Female_Cast_Only as
(
select year, count(mid) Female_Cast_Only_Movies
from
(select mid,Count,year
from gender_count
where gender='Female') g1
left join
(select mid
from gender_count
where gender = 'Male' or gender = 'None') g2
using (mid)
where g2.mid is null
group by year
)

select f.year year , cast(f.Female_Cast_Only_Movies as float)/cast(count(*) as float) Percentage_Female_Only_Movie , count(*) Total_Movies
from
Female_Cast_Only f,
movie m
where f.year = m.year
group by f.year

'''

pd.read_sql_query(query5b,conn)

In [ ]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = query5b
grader_5b(query5b)

   year  Percentage_Female_Only_Movie  Total_Movies
0  1939                      0.500000             2
1  1999                      0.015152            66
2  2000                      0.015625            64
3  2018                      0.009615           104
CPU times: user 297 ms, sys: 17.5 ms, total: 314 ms
Wall time: 313 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast.

In [ ]:
query6 = '''
select m.title, count(distinct(m_c.pid)) count
from
movie m
inner join m_cast m_c
using (mid)
group by mid
order by count desc
'''

pd.read_sql_query(query6,conn)

In [ ]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = query6
grader_6(query6)

                        title  count
0               Ocean's Eight    238
1                    Apaharan    233
2                        Gold    215
3             My Name Is Khan    213
4  Captain America: Civil War    191
5                    Geostorm    170
6                     Striker    165
7                        2012    154
8                      Pixels    144
9       Yamla Pagla Deewana 2    140
CPU times: user 151 ms, sys: 6.48 ms, total: 158 ms
Wall time: 160 ms


## Q7 --- Find the decade D with the largest number of films and the total number of films in D

In [ ]:
query7a = '''
select m.year Movie_Year, count(m.mid) Total_Movies
from
movie m
group by m.year
'''

pd.read_sql_query(query7a,conn)

In [ ]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = query7a
grader_7a(query7a)

# using the above query, you can write the answer to the given question

  Movie_Year  Total_Movies
0       1931             1
1       1936             3
2       1939             2
3       1941             1
4       1943             1
5       1946             2
6       1947             2
7       1948             3
8       1949             3
9       1950             2
CPU times: user 4.9 ms, sys: 0 ns, total: 4.9 ms
Wall time: 5.83 ms


In [ ]:
query7b = '''
with table7a as
(
select m.year Movie_Year, count(m.mid) Total_Movies
from
movie m
group by m.year
)
select t1.Movie_Year, t1.Total_Movies, t2.Movie_Year, t2.Total_Movies
from
table7a t1
join
table7a t2
on t2.Movie_Year - t1.Movie_Year <=9 and t1.Movie_Year<=t2.Movie_Year

'''

pd.read_sql_query(query7b,conn)

In [ ]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b =query7b
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

  Movie_Year  Total_Movies Movie_Year  Total_Movies
0       1931             1       1931             1
1       1931             1       1936             3
2       1931             1       1939             2
3       1936             3       1936             3
4       1936             3       1939             2
5       1936             3       1941             1
6       1936             3       1943             1
7       1939             2       1939             2
8       1939             2       1941             1
9       1939             2       1943             1
CPU times: user 6.93 ms, sys: 0 ns, total: 6.93 ms
Wall time: 6.97 ms


In [ ]:
query7 = '''
with table7a as
(
select m.year Movie_Year, count(m.mid) Total_Movies
from
movie m
group by m.year
),
max_decade as
(
select sum(t2.Total_Movies) Decade_Movie_Count, t1.Movie_Year Decade
from
table7a t1
join
table7a t2
on t2.Movie_Year - t1.Movie_Year <=9 and t1.Movie_Year<=t2.Movie_Year
group by t1.Movie_Year
)

select max(d.Decade_Movie_Count) Decade_Movie_Count , d.Decade
from
max_decade d
'''

pd.read_sql_query(query7,conn)

In [ ]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = query7
grader_7(query7)

   Decade_Movie_Count Decade
0                1203   2008
CPU times: user 6.45 ms, sys: 0 ns, total: 6.45 ms
Wall time: 6.24 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
query8a = '''
select m_c.pid actor, m_d.pid director, count(distinct(m.mid)) movies
from
movie m
inner join m_director m_d
using (mid)
inner join m_cast m_c
using (mid)
group by m_d.pid,m_c.pid
'''

pd.read_sql_query(query8a,conn)

In [ ]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = query8a
grader_8a(query8a)

# using the above query, you can write the answer to the given question

       actor   director  movies
0  nm0000027  nm0000180       1
1  nm0001114  nm0000180       1
2  nm0001919  nm0000180       1
3  nm0006762  nm0000180       1
4  nm0030062  nm0000180       1
5  nm0038970  nm0000180       1
6  nm0051856  nm0000180       1
7  nm0085966  nm0000180       1
8  nm0097889  nm0000180       1
9  nm0125497  nm0000180       1
CPU times: user 337 ms, sys: 19.6 ms, total: 357 ms
Wall time: 362 ms


In [ ]:
query8 = '''

with actor_dir as
(
select m_c.pid actor, m_d.pid director, count(m.mid) movies
from
movie m
inner join m_director m_d
using (mid)
inner join m_cast m_c
using (mid)
group by m_c.pid,m_d.pid
),

yash_chopra_id as
(
select p.pid id
from
person p
where p.name='Yash Chopra'
),

actor_yash_copra_movies as
(
select a_d.actor yc_actor, a_d.movies count_yc
from yash_chopra_id yc_id, actor_dir a_d
where a_d.director = yc_id.id
),

actor_max_movies as
(
select a_yc.yc_actor actor_max, a_d.director dir, max(a_d.movies) count_max
from
actor_dir a_d, actor_yash_copra_movies a_yc
where a_d.actor = a_yc.yc_actor
group by a_d.actor
),

actor_yc_max as
(
select a_yc.yc_actor actor_final, a_yc.count_yc count_final
from
actor_yash_copra_movies a_yc 
join
actor_max_movies a_m 
on a_yc.yc_actor = a_m.actor_max
where a_yc.count_yc >= a_m.count_max
)

select p.name , a.count_final
from
actor_yc_max a 
join person p
on a.actor_final = p.pid
order by a.count_final desc
'''

pd.read_sql_query(query8,conn)

In [ ]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = query8
grader_8(query8)

               Name  count_final
0       Jagdish Raj           11
1  Manmohan Krishna           10
2          Iftekhar            9
3     Shashi Kapoor            7
4     Rakhee Gulzar            5
5    Waheeda Rehman            5
6          Ravikant            4
7    Achala Sachdev            4
8       Neetu Singh            4
9     Leela Chitnis            3
(245, 2)
CPU times: user 859 ms, sys: 39.2 ms, total: 899 ms
Wall time: 900 ms


## Q9 --- Return all actors whose Shahrukh number is 2.

In [ ]:
query9a = '''

with srk_id as
(
select p.pid id
from person p
where p.name = 'Shah Rukh Khan'
),

srk_movies as
( 
select m_c.mid movies
from srk_id s, m_cast m_c
where m_c.pid = s.id
)

select distinct(m_c.pid) c1
from srk_movies s_m, m_cast m_c,srk_id s
where m_c.mid = s_m.movies and m_c.pid != s.id

'''

pd.read_sql_query(query9a,conn)

In [ ]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = query9a
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

          c1
0  nm0001934
1  nm0008279
2  nm0012881
3  nm0066455
4  nm0080251
5  nm0149822
6  nm0160463
7  nm0221222
8  nm0416077
9  nm0420090
(2382, 1)
CPU times: user 186 ms, sys: 7.57 ms, total: 194 ms
Wall time: 193 ms


In [ ]:
query9 = '''

with srk_id as
(
select p.pid id
from person p
where p.name = 'Shah Rukh Khan'
),

srk_movies as
( 
select m_c.mid movies
from srk_id s, m_cast m_c
where m_c.pid = s.id
),

cast_1 as
(
select distinct(m_c.pid) c1
from srk_movies s_m, m_cast m_c,srk_id s
where m_c.mid = s_m.movies and m_c.pid != s.id
),

movies_cast_1 as
(
select m_c.mid mid1
from
cast_1 c_1, m_cast m_c
where m_c.pid = c_1.c1
),

cast_2 as
(
select distinct(m_c.pid) c2
from movies_cast_1 m_c1, m_cast m_c
where m_c.mid = m_c1.mid1
),

final_id as
(
select c_2.c2 id
from srk_id s,
cast_2 c_2
left join cast_1 c_1
on c_2.c2 = c_1.c1
where c_1.c1 is Null and c_2.c2 != s.id
)

select p.name
from person p, final_id a
where a.id = p.pid
'''

pd.read_sql_query(query9,conn)

In [ ]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = query9
grader_9(query9)

                    Name
0           Freida Pinto
1            Rohan Chand
2           Damian Young
3        Waris Ahluwalia
4  Caroline Christl Long
5          Rajeev Pahuja
6      Michelle Santiago
7        Alicia Vikander
8           Dominic West
9         Walton Goggins
(25698, 1)
CPU times: user 850 ms, sys: 23.2 ms, total: 873 ms
Wall time: 868 ms
